# GBM (Gradient Boosting Machine)

 `-` 다음은 [wikipedia](https://en.wikipedia.org/wiki/Gradient_boosting)의 Gradient Boosting Algorithm이다. 다음의 수식을 먼저 읽고 최대한 이해하려고 노력해보자

## Algorithm

1. **initialize model with a constant value:**\
    $F_0(x) = \underset{\gamma}{\arg\min}\sum_{i=1}^n L(y_i, \gamma)$

 2. **For m = 1 to M**: 
    1. Compute so-called pseudo-residuals
       $r_{im} = - \Bigg[\displaystyle\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\Bigg]_{F(x) = F_{m-1}(x)}$ for $i = 1, ... ,n$
    2. Fit a base learner (or weak learner, e.g. tree) closed under scaling $h_m(x)$ to pseuo-residuals, i.e. train it using the training set $\{(x_i, r_{im})\}_{i=1}^{n}$
    3. Compute multiplier $\gamma_{m}$ by solving the following one-dimensional optimization problem:\
       $\gamma_m = \underset{\gamma}{\arg\min}\sum_{i=1}^n L(y_i, F_{m-1}(x_i) + \gamma h_m(x_i))$
    4. Update the model:\
       $F_m(x) = F_{m-1}(x) + \gamma_m h_m(x)$
    
3. **Output $\hat{f(x)} = F_M(x)$**

   
       

수식으로만 완전히 이해하기가 어렵기 때문에 간단한 예시와 `pandas dataframe`을 이용하여 설명해보겠습니다.\
다음과 같은 데이터가 있다고 가정해보겠습니다.

In [1]:
import pandas as pd
data = pd.DataFrame({'공부시간(m)':[150, 120, 60, 80], '성별':['여','남','남', '여'], '수학점수':[90, 65, 55, '???']})
data

,공부시간(m),성별,수학점수
0,150,여,90
1,120,남,65
2,60,남,55
3,80,여,???


다음과 같은 데이터가 있다고 할 때, 저희는 `공부시간(m)`을 통하여 `수학점수`를 예측하고 싶습니다.

#### **1단계**

1. **initialize model with a constant value:**\
    $F_0(x) = \underset{\gamma}{\arg\min}\sum_{i=1}^n L(y_i, \gamma)$


1. 초기모델을 상수로 정의합니다. 여기서 상수는 $y_i$와 $\gamma$입니다.
2. $y_i$는 수학점수를 의미하고 $\gamma $는 초기 예측값입니다.
    - $y_1$ = 90,  $y_2$ = 65, $y_3$ = 60
3. 여기서의 Loss Function $L(y_i, \gamma) = \displaystyle\frac1n \sum_{i=1}^{n} (y_i - \gamma)^2$입니다. (잘 모르시는 분들은 MSE 학습하시면 좋을 것 같습니다.)
4. 예시의 수치를 대입하면\
$L(y_i, \gamma) = \displaystyle\frac13 \sum_{i=1}^{3} (y_i - \gamma)^2 =  \displaystyle\frac13(90 - \gamma)^2 + \displaystyle\frac13(65 - \gamma)^2 + \displaystyle\frac13(55 - \gamma)^2$입니다.
5. $L(y_i, \gamma)$가 최소가 되는 $\gamma$를 찾아야 하므로
$\displaystyle\frac{\partial L}{\partial \gamma} = \displaystyle\frac23(\gamma - 90) + \displaystyle\frac23(\gamma - 65) + \displaystyle\frac23(\gamma - 55) = \displaystyle\frac23(3\gamma - 210)$입니다.
6. $\displaystyle\frac{\partial L}{\partial \gamma} = 0$이 되어야 하므로 $\gamma$ = 70입니다.

#### **2단계**


 2. **For $m$ = 1 to $M$**: 
    1. Compute so-called pseudo-residuals
       $r_{im} = - \Bigg[\displaystyle\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\Bigg]_{F(x) = F_{m-1}(x)}$ for $i = 1, ... ,n$
    2. Fit a base learner (or weak learner, e.g. tree) closed under scaling $h_m(x)$ to pseuo-residuals, i.e. train it using the training set $\{(x_i, r_{im})\}_{i=1}^{n}$
    

$r_{im}$은 가짜 잔차(pseudo-residuals)라고 부릅니다.

$L(y_i, F(x_i)) = \displaystyle\frac1n \sum_{i=1}^{n}(y_i - F(x_i))^2$이므로\
$r_{im} = -\Bigg[\displaystyle\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \Bigg]_{F(x)=F_{m-1}(x)} =\displaystyle\frac2n\sum_{i=1}^{n}(y_i - F_{m-1}(x_i))$입니다. 정리하면,\
$r_{im} = \displaystyle\frac2n\sum_{i=1}^{n}(y_i - F_{m-1}(x_i))$입니다.

1. 여기서 $M$은 예측기의 개수, $n$은 데이터의 개수입니다.
2. For $m$ = 1 to $M$의 의미는 M개의 예측기에 대해서 특정 작업을 수행한다는 의미입니다.
3. for $i$ = $1, ... ,n$은 $n$개의 설명변수 $x$에 대해서 $x_1$부터 $x_n$까지 특정 작업을 수행한다는 의미입니다.
4. $r_{im}$은 가짜 잔차(pseudo-residuals)라고 부릅니다.
5. $(m-1)$번째 학습한 모델은 $F_{m-1}(x)$입니다. 이떄의 Loss Function을 $F_{m-1}(x)$로 미분한 것이 $-r_{im}$입니다.
6. 식을 확인해보면 실제 잔차$(y_i - F_{m-1}(x))$와 유사해서 가짜 잔차로 불립니다.
7. 이때 $h_{m}(x)$는 가짜 잔차를 학습합니다.
8. 즉, $h_{m}(x) = \widehat{y-F_{m-1}(x)}$

#### **3단계**

2. **For m = 1 to M**:\
    C. Compute multiplier $\gamma_{m}$ by solving the following one-dimensional optimization problem:\
       $\gamma_m = \underset{\gamma}{\arg\min}\sum_{i=1}^n L(y_i, F_{m-1}(x_i) + \gamma h_m(x_i))$
    